In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from konlpy.tag import Okt
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
%matplotlib inline

In [2]:
DATA_IN_PATH = './data_in/'
print("파일 크기 : ")
for file in os.listdir(DATA_IN_PATH):
    if 'csv' in file :
        print(file.ljust(30) + str(round(os.path.getsize(DATA_IN_PATH + file) / 1000000, 2)) + 'MB')

파일 크기 : 
real_refined_data.csv         1.12MB
real_test.csv                 0.75MB
real_test_data.csv            0.56MB
real_test_label.csv           0.02MB
real_train.csv                0.76MB
real_train_data.csv           0.56MB


In [3]:
train_data = pd.read_csv(DATA_IN_PATH+'real_train_data.csv', header = 0)
test_data = pd.read_csv(DATA_IN_PATH+'real_test_data.csv', header = 0)
train_data.head()
test_data.head()

,sentence,label
0,그럼 회의를 열어서 신입을 모집할지 경력직을 모집할지 정합시다,0
1,상황 해결을 위해 이번 분기에 경력직을 모집하는 건 어떨까요,0
2,다양한 경험을 가진 분이 온다면 상황을 해결하는 데 큰 도움을 줄 것 같습니다,0
3,그럼 다음 달에 홈페이지와 신문에 모집 공고를 띄울까요,0
4,최대한 빨리 공고를 띄워서 이번 달 안에 선발 부탁드립니다,0


In [4]:
print("train 긍정 리뷰 개수: {}".format(train_data['label'].value_counts()[0]))
print("train 부정 리뷰 개수: {}".format(train_data['label'].value_counts()[1]))
print("test 긍정 리뷰 개수: {}".format(test_data['label'].value_counts()[0]))
print("test 부정 리뷰 개수: {}".format(test_data['label'].value_counts()[1]))

train 긍정 리뷰 개수: 3252
train 부정 리뷰 개수: 3252
test 긍정 리뷰 개수: 3253
test 부정 리뷰 개수: 3253


In [5]:
def preprocessing(sentence, okt, remove_stopwords = False, stop_words = []):
    # 함수의 인자는 다음과 같다.
    # sentence : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    sentence_text = re.sub("[ㄱ-ㅎㅏ-ㅣ\\s]+", "", sentence) 
    
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    wd_sentence = okt.morphs(sentence_text, stem=True)
    
    if remove_stopwords:
        
        # 불용어 제거(선택적)
        wd_sentence = [token for token in wd_sentence if not token in stop_words]
        
   
    return wd_sentence

In [6]:
stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
okt = Okt()
clean_train_sentence = []

for review in train_data['sentence']:
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_train_sentence.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_train_sentence.append([])  #string이 아니면 비어있는 값 추가
clean_train_sentence[:4]

clean_train_df = pd.DataFrame({'sentence':clean_train_sentence,'label':train_data['label']})

In [7]:
#test_data = pd.read_csv(DATA_IN_PATH + 'testdata.csv', headear = 0)
#test_data = pd.read_csv(DATA_IN_PATH + 'testdata.csv', names=['sentence'] )

clean_test_sentence = []

for review in test_data['sentence']:
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_test_sentence.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_test_sentence.append([])  #string이 아니면 비어있는 값 추가
        
clean_test_onlysentence_df = pd.DataFrame({'sentence':clean_test_sentence})
clean_test_onlylabel_df = pd.DataFrame({'label':test_data['label']})

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_sentence) ###only train data만 단어사전에 
train_sequences = tokenizer.texts_to_sequences(clean_train_sentence)
test_sequences = tokenizer.texts_to_sequences(clean_test_sentence)

word_vocab = tokenizer.word_index # 단어 사전 형태
# word_vocab["<PAD>"] = 0

MAX_SEQUENCE_LENGTH = 30 #문장 최대 길이, 중간값 지정

train_inputs = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') #train data벡터화
train_labels = np.array(train_data['label'])
test_inputs = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') #test data 벡터화
test_labels = np.array(test_data['label'])

In [9]:
DATA_IN_PATH = './data_in/'
from collections import OrderedDict

TRAIN_INPUT_DATA = 'real_train_input.npy'
TRAIN_LABEL_DATA = 'real_train_label.npy'
TRAIN_CLEAN_DATA = 'real_train.csv'

TEST_INPUT_DATA = 'real_test_input.npy'
TEST_CLEAN_DATA = 'real_test.csv'
TEST_CLEAN_LABEL = 'real_test_label.csv'
TEST_LABEL_DATA = 'real_test_label.npy'

DATA_CONFIGS = 'real_data_configs.json'

# data_configs = {}
data_configs = OrderedDict()

data_configs['vocab'] = word_vocab
#data_configs['vocab_size'] = len(word_vocab) # vocab size 추가
data_configs['vocab_size'] = len(word_vocab)+1

import os
# 저장하는 디렉토리가 존재하지 않으면 생성
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)

# 전처리 된 학습 데이터를 넘파이 형태로 저장
np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_labels)

# 전처리 된 학습 데이터를 csv 형태로 저장
clean_train_df.to_csv(DATA_IN_PATH +TRAIN_CLEAN_DATA, index = False)
clean_test_onlysentence_df.to_csv(DATA_IN_PATH +TEST_CLEAN_DATA, index = False)
clean_test_onlylabel_df.to_csv(DATA_IN_PATH + TEST_CLEAN_LABEL, index = False )

# 전처리 된 테스트 데이터를 넘파이 형태로 저장
np.save(open(DATA_IN_PATH + TEST_INPUT_DATA, 'wb'), test_inputs)
np.save(open(DATA_IN_PATH + TEST_LABEL_DATA, 'wb'), test_labels)

# 데이터 사전을 json 형태로 저장
#json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w',encoding='UTF-8'), ensure_ascii=False)

#json 파일 생성
with open(DATA_IN_PATH + DATA_CONFIGS, 'w', encoding='UTF-8') as f:
    json_data = json.dumps(data_configs)
    json_decode_data = json_data.encode('utf8').decode('unicode-escape')
    f.write(json_decode_data)